# BlackCat Ransomware
> Analysis of new BlackCat ransomware  with encrypted config

- toc: true 
- badges: true
- categories: [blackcat,ransomware,malware,python,dumpulator,emulation]

> twitter: https://twitter.com/pancak3lullz/status/1504208693277212674

## Overview

Sample `6dd995d896a9a593b2c48d09da60bd83866d8577273f36d38788d83ad8173e68`

**References**
- Sample is on [MalwareBazaar](https://bazaar.abuse.ch/sample/6dd995d896a9a593b2c48d09da60bd83866d8577273f36d38788d83ad8173e68/)
- Old config extracto from f0wl on [blackCatConf](https://github.com/f0wl/blackCatConf) 
- Paloalto [Threat Assessment: BlackCat Ransomware](https://unit42.paloaltonetworks.com/blackcat-ransomware/)
- Talos [From BlackMatter to BlackCat: Analyzing two attacks from one affiliate](https://blog.talosintelligence.com/2022/03/from-blackmatter-to-blackcat-analyzing.html)

### Rust Reverse Engineering
BlackCat is written in Rust and is very time consuming to reverse engineer. To speed up the process we used a combination of debugging and some python scripts courtesy of @fwosar.

### The Config
There is a new BlackCat ransomware sample out and it the config is now protected using a command line supplied ACCCESS_TOKEN. The token is used to generate an AES key which is then used to decrypt the encrypted config.





### Notes

**Ransomware options**
```
USAGE:
     [OPTIONS] [SUBCOMMAND]

OPTIONS:
        --access-token <ACCESS_TOKEN>               Access Token
        --bypass <BYPASS>...
        --child                                     Run as child process
        --drag-and-drop                             Invoked with drag and drop
        --drop-drag-and-drop-target                 Drop drag and drop target batch file
        --extra-verbose                             Log more to console
    -h, --help                                      Print help information
        --log-file <LOG_FILE>                       Enable logging to specified file
        --no-net                                    Do not discover network shares on Windows
        --no-prop                                   Do not self propagate(worm) on Windows
        --no-prop-servers <NO_PROP_SERVERS>...      Do not propagate to defined servers
        --no-vm-kill                                Do not stop VMs on ESXi
        --no-vm-kill-names <NO_VM_KILL_NAMES>...    Do not stop defined VMs on ESXi
        --no-vm-snapshot-kill                       Do not wipe VMs snapshots on ESXi
        --no-wall                                   Do not update desktop wallpaper on Windows
    -p, --paths <PATHS>...                          Only process files inside defined paths
        --propagated                                Run as propagated process
        --ui                                        Show user interface
    -v, --verbose                                   Log to console
```

**Set desktop ransom note**
`\\All Usersdeploy_note_and_image_for_all_users=`

**Mutex**
`Local\\RustBacktraceMutex`


**Batch file name**
`drag-and-drop-target.bat`


**Batch file contents (encrypted)**
```
@ECHO OFF
SETLOCAL
SET allargs=%*
"${EXECUTABLE}" --access-token ${ACCESS_TOKEN} --drag-and-drop -p %allargs%
```

### Propogation  
The ransomware contains an embedded encrypted copy of [psExec](https://docs.microsoft.com/en-us/sysinternals/downloads/psexec). This is likely used as part of their propagation (worm). The ransomware can be executed via the `drag-and-drop-target.bat` batch files. These are a useful artifact to recover during IR as they will contain the ACCESS_TOKEN which can be used to decrypt the config. 




### Entropy Fider

To help identify high entropy sections of the binary (encrypted data) we used this nice script from @fwosar

In [4]:
# Entropy find script by @fwosar

def report_entropies(file_path):
    import rolling
    from pathlib import Path
    data = Path(file_path).read_bytes()
 
    entropies = rolling.Entropy(data, 512)
    high_entropy_start = None
    for i, entropy in enumerate(entropies):
        if entropy > 7 and not high_entropy_start:
            high_entropy_start = i + 512
        if entropy < 7 and high_entropy_start:
            print(f"Found high entropy section starting at {hex(high_entropy_start)} up to {hex(i + 512)}. Totalling {(i + 512) - high_entropy_start} bytes.")
            high_entropy_start = None
    if high_entropy_start:
        print(f"Found high entropy section starting at {hex(high_entropy_start)} up to {hex(len(data))}. Totalling {len(data) - high_entropy_start} bytes.")